Kickstart the simulation.

In [1]:
%pip install --extra-index-url https://test.pypi.org/simple ../EnergyPlus-OOEP/
%pip install gymnasium ray[rllib] torch tqdm 
%pip install /home/user@AD/lab/EnergyPlus-Datasets/

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com, https://test.pypi.org/simple
Processing /home/user@AD/lab/EnergyPlus-OOEP
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for energyplus-ooep: filename=energyplus_ooep-0.0.post1.dev16+ge7e43b9-py3-none-any.whl size=31682 sha256=5355c2a2f8b5e5e1cf66bc378cf2a79ae036e464520da5f606f8784e8612777b
  Stored in directory: /home/user@AD/.cache/pip/wheels/16/f9/7c/4f4cbae4f9bac45a57e6927307e3d5abdc8afb7240c9c634ff
Successfully built energyplus-ooep
  Attempting uninstall: energyplus-ooep
    Found existing installation: energyplus-ooep 0.0.post1.dev16+ge7e43b9
    Uninstalling energyplus-ooep-0.0.post1.dev16+ge7e43b9:
      Successfully uninstalled energyplus-ooep-0.0.post1.dev16+ge7e43b9
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://py

In [2]:
#import logging as _logging_
#_logging_.basicConfig(level='INFO')
#from energyplus.ooep.addons.logging import LogProvider

from energyplus.ooep.addons.progress import ProgressProvider

from energyplus.ooep import (
    Simulator,
    Model,
    Weather,
    Report,
)

from energyplus.dataset.basic import dataset as _epds_

simulator = Simulator().add(
    ProgressProvider(),
    #LogProvider(),
)

simulator.awaitable.run_forever(
    input=Simulator.InputSpecs(
        model=(
            _epds_.models / 'ASHRAE901_OfficeLarge_STD2019_Denver_Chiller205_Detailed.idf'
        ),
        weather=(_epds_.weathers / 'USA_CO_Denver-Aurora-Buckley.AFB.724695_TMY3.epw'),
    ),
    output=Simulator.OutputSpecs(
        #report=('/tmp/ooep-report-9e1287d2-8e75-4cf5-bbc5-f76580b56a69'),
    ),
    options=Simulator.RuntimeOptions(
        #design_day=True,
    ),
)

/home/user@AD/lab/EnergyGym/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/100 [00:00<?, ?it/s]

<Task pending name='Task-5' coro=<Engine.run_forever() running at /home/user@AD/lab/EnergyGym/.venv/lib/python3.11/site-packages/energyplus/ooep/utils/awaitables.py:68>>

method 1

In [3]:
import energyplus.ooep as _ooep_

try:
    obs = simulator.variables.getdefault(
        _ooep_.OutputVariable.Ref(
            type='People Air Temperature',
            key='CORE_MID',
        )
    ).value

    simulator.variables.getdefault(
        _ooep_.Actuator.Ref(
            type='Zone Temperature Control',
            control_type='Heating Setpoint',
            key='CORE_MID',
        )
    ).value = 16
    actuator = simulator.variables.getdefault(
        _ooep_.Actuator.Ref(
            type='Zone Temperature Control',
            control_type='Heating Setpoint',
            key='CORE_MID',
        )
    ).value 
    print(obs,actuator)
except _ooep_.TemporaryUnavailableError:
    pass

 65%|██████▌   | 65.0/100 [01:13<00:03,  9.21it/s, Initializing New Environment Parameters]                                            

method 2

In [4]:
import numpy as _numpy_
import gymnasium as _gymnasium_

from energyplus.ooep.addons.rl import (
    VariableBox,
    SimulatorEnv,
)

from energyplus.ooep import (
    Actuator,
    OutputVariable,
)
from energyplus.ooep.addons.rl.gymnasium import ThinEnv



simulator.add(
    thinenv := ThinEnv(
        action_space=_gymnasium_.spaces.Dict({
                    'thermostat': VariableBox(
                        low=15., high=16.,
                        dtype=_numpy_.float32,
                        shape=(),
                    ).bind(Actuator.Ref(
                        type='Zone Temperature Control',
                        control_type='Heating Setpoint',
                        key='CORE_MID',
                    ))
                }),    
        observation_space=_gymnasium_.spaces.Dict({
            'temperature': VariableBox(
                low=-_numpy_.inf, high=+_numpy_.inf,
                dtype=_numpy_.float32,
                shape=(),
            ).bind(OutputVariable.Ref(
                type='People Air Temperature',
                key='CORE_MID',
            )),
        }),
    )
)


2024-04-21 08:07:28,227	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-04-21 08:07:29,142	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [5]:
# some test event
#simulator.events.on('message', lambda event: print('🥳', event))

In [6]:
def handler(__event):
    global thinenv
    obs = thinenv.observe()
    # ... use the observation
    #act = thinenv.act({'thermostat': 16. })
    thinenv.act(
        thinenv.action_space.sample()
    )

# event-driven control example
simulator.events.on('begin_zone_timestep_after_init_heat_balance', handler)